# Project Import Debugging

This notebook will help diagnose and fix import errors in the exoplanet transit detection project. It will systematically check:

1. Module import structure
2. Python path configuration 
3. Relative vs absolute imports
4. File locations and project structure

In [ ]:
import sys
import os

# Display the current Python path
print("Current Python path:")
for p in sys.path:
    print(f"  {p}")
    
# Display the current working directory
print(f"\nCurrent working directory: {os.getcwd()}")

# Check if important project directories exist
project_root = os.path.abspath(".")
shared_dir = os.path.join(project_root, "shared")
data_utils_dir = os.path.join(shared_dir, "data_utils")

print(f"\nProject root exists: {os.path.exists(project_root)}")
print(f"shared/ directory exists: {os.path.exists(shared_dir)}")
print(f"shared/data_utils/ directory exists: {os.path.exists(data_utils_dir)}")

# Check if we can import the shared.data_utils module directly
print("\nAttempting to import shared.data_utils directly:")
try:
    import shared.data_utils
    print("✓ Success! The module can be imported")
except ImportError as e:
    print(f"✗ Import error: {e}")

## Testing Import Approaches

Let's try different approaches to importing the shared.data_utils module, including:
1. Explicit path additions
2. Different relative path references
3. Absolute imports

In [ ]:
# Approach 1: Add project root to Python path
import sys
import os

project_root = os.path.abspath(".")
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added {project_root} to sys.path")
else:
    print(f"{project_root} already in sys.path")

try:
    from shared.data_utils import load_flux_from_fits, normalize_flux
    print("✓ Import successful with approach 1 (adding project root to path)")
except ImportError as e:
    print(f"✗ Import failed with approach 1: {e}")

# Approach 2: Using explicit absolute imports
try:
    import shared.data_utils.fits_loader
    print("✓ Explicit absolute import successful")
except ImportError as e:
    print(f"✗ Explicit absolute import failed: {e}")

In [ ]:
# Approach 3: Using relative imports from other locations
# This simulates running the notebook from different locations

# Simulate running from exoplanet-transit-detection/notebooks directory
notebook_dir = os.path.join(project_root, "projects", "exoplanet-transit-detection", "notebooks")
print(f"Testing as if running from: {notebook_dir}")

# Attempt relative import from simulated notebook directory
potential_paths = [
    os.path.join(notebook_dir, "..", ".."),  # ../../ from notebook dir
    os.path.join(notebook_dir, "..", "..", ".."),  # ../../../ from notebook dir
    os.path.join(notebook_dir, ".."),  # ../ from notebook dir
]

for path in potential_paths:
    abs_path = os.path.abspath(path)
    print(f"\nTesting with sys.path addition: {abs_path}")
    
    # Create a clean sys.path
    original_path = sys.path.copy()
    sys.path = [p for p in sys.path if p != project_root]  # Remove the project root we added earlier
    sys.path.insert(0, abs_path)
    
    try:
        # Clear any previously imported modules to force a fresh import
        if 'shared.data_utils' in sys.modules:
            del sys.modules['shared.data_utils']
        if 'shared' in sys.modules:
            del sys.modules['shared']
        
        # Try to import
        from shared.data_utils import load_flux_from_fits
        print(f"✓ Success with path: {abs_path}")
    except ImportError as e:
        print(f"✗ Failed with path: {abs_path} - Error: {e}")
    finally:
        # Restore original path
        sys.path = original_path

## Examining Module Structure

Let's examine the structure of the `shared.data_utils` module to understand what's being imported and how it's organized.

In [ ]:
# Add project root to path if not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Check the structure of the module files
data_utils_init = os.path.join(project_root, "shared", "data_utils", "__init__.py")
fits_loader_path = os.path.join(project_root, "shared", "data_utils", "fits_loader.py")
data_processing_path = os.path.join(project_root, "shared", "data_utils", "data_processing.py")

print(f"__init__.py exists: {os.path.exists(data_utils_init)}")
print(f"fits_loader.py exists: {os.path.exists(fits_loader_path)}")
print(f"data_processing.py exists: {os.path.exists(data_processing_path)}")

# Display the content of __init__.py
print("\nContents of __init__.py:")
try:
    with open(data_utils_init, 'r') as f:
        print(f.read())
except Exception as e:
    print(f"Error reading file: {e}")

# Check imports from fits_loader.py
print("\nChecking imports in fits_loader.py:")
try:
    import shared.data_utils.fits_loader
    print("Module imports successfully")
    
    # List available functions in the module
    functions = [f for f in dir(shared.data_utils.fits_loader) 
                if not f.startswith('_') and callable(getattr(shared.data_utils.fits_loader, f))]
    print(f"Available functions: {functions}")
except Exception as e:
    print(f"Error importing module: {e}")

## Testing Notebook Import Context

Now let's simulate the exact import pattern used in the exoplanet-transit-detection notebook. We'll create a test function that replicates the same import pattern and error handling.

In [ ]:
def test_notebook_imports(path_addition=None):
    """Test imports with the same pattern as the notebook"""
    # Create a clean environment to test
    import sys
    import os
    import importlib
    
    # Store original sys.path
    original_path = sys.path.copy()
    
    # Remove any existing modules that might interfere
    for module in list(sys.modules.keys()):
        if module.startswith('shared'):
            del sys.modules[module]
    
    try:
        # Add the path that would be added in the notebook
        if path_addition:
            if isinstance(path_addition, str):
                path_addition = [path_addition]
            for path in path_addition:
                abs_path = os.path.abspath(path)
                print(f"Adding path: {abs_path}")
                sys.path.insert(0, abs_path)
        
        # Try the import
        print("Attempting import: from shared.data_utils import load_flux_from_fits, normalize_flux")
        from shared.data_utils import load_flux_from_fits, normalize_flux
        print("✓ Import successful!")
        
        # Test if functions work
        print("\nTesting if imported functions are callable:")
        try:
            print(f"load_flux_from_fits is callable: {callable(load_flux_from_fits)}")
            print(f"normalize_flux is callable: {callable(normalize_flux)}")
        except Exception as e:
            print(f"Error testing functions: {e}")
            
        return True
    except ImportError as e:
        print(f"✗ Import failed: {e}")
        return False
    finally:
        # Restore original sys.path
        sys.path = original_path

# Test 1: Simulate the notebook's import pattern
print("Simulation 1: Add '../../' to sys.path (as done in the notebook)")
test_notebook_imports(os.path.join(project_root, "projects", "exoplanet-transit-detection", "..", ".."))

# Test 2: Just add the project root directly
print("\nSimulation 2: Add project root directly to sys.path")
test_notebook_imports(project_root)

## Simulating Notebook Running Context

Let's create functions that simulate running the notebook in its actual location, with the proper directory structure and imports.

In [ ]:
import os
import sys
import traceback

def simulate_notebook_environment():
    """
    Simulate the environment of the notebook running from its actual location.
    This helps diagnose issues related to running the notebook from different contexts.
    """
    # Location of the actual notebook
    notebook_path = os.path.join(project_root, "projects", "exoplanet-transit-detection", "notebooks", "explore_tess.ipynb")
    notebook_dir = os.path.dirname(notebook_path)
    
    print(f"Notebook file exists: {os.path.exists(notebook_path)}")
    print(f"Notebook directory: {notebook_dir}")
    
    # Store original working directory and sys.path
    original_cwd = os.getcwd()
    original_path = sys.path.copy()
    
    try:
        # Change to the notebook directory
        os.chdir(notebook_dir)
        print(f"Changed working directory to: {os.getcwd()}")
        
        # Add the path as the notebook does
        parent_parent_dir = os.path.abspath(os.path.join("..",".."))
        sys.path.insert(0, parent_parent_dir)
        print(f"Added to sys.path: {parent_parent_dir}")
        
        # Try the import
        print("\nAttempting import as the notebook would:")
        try:
            # Clear any existing modules
            if 'shared' in sys.modules:
                del sys.modules['shared']
            if 'shared.data_utils' in sys.modules:
                del sys.modules['shared.data_utils']
                
            from shared.data_utils import load_flux_from_fits, normalize_flux
            print("✓ Import successful!")
            
            # Test calling a function
            print("\nTest calling a function:")
            try:
                # Don't actually call the function as it requires a FITS file
                # Just check if it's callable
                print(f"load_flux_from_fits is callable: {callable(load_flux_from_fits)}")
            except Exception as e:
                print(f"Error testing function: {e}")
                
        except ImportError as e:
            print(f"✗ Import failed: {e}")
            print("\nTraceback:")
            traceback.print_exc()
    finally:
        # Restore original state
        os.chdir(original_cwd)
        sys.path = original_path
        print(f"\nRestored working directory to: {os.getcwd()}")

# Run the simulation
simulate_notebook_environment()

## Testing Notebook With Debug Instrumentation

Let's create a version of the cell that's causing issues, but with added debugging to help identify what's going wrong.

In [ ]:
def debug_import_with_instrumentation():
    """
    Create a version of the problematic import code with extensive debugging information.
    """
    import sys
    import os
    import traceback
    
    # Restore original environment
    original_path = sys.path.copy()
    
    try:
        # Clean up any existing imports
        for module in list(sys.modules.keys()):
            if module.startswith('shared'):
                del sys.modules[module]
        
        print("Current sys.path BEFORE modifications:")
        for p in sys.path:
            print(f"  {p}")
        
        # Add project root to sys.path - similar to what the notebook does
        project_root = os.path.abspath(".")
        sys.path.insert(0, project_root)
        print(f"\nAdded {project_root} to sys.path")
        
        print("\nCurrent sys.path AFTER modifications:")
        for p in sys.path:
            print(f"  {p}")
            
        # Check module existence
        print("\nChecking if module files exist:")
        module_path = os.path.join(project_root, "shared", "data_utils")
        init_file = os.path.join(module_path, "__init__.py")
        print(f"Module directory exists: {os.path.exists(module_path)}")
        print(f"__init__.py exists: {os.path.exists(init_file)}")
        
        # Now try the import with verbose error handling
        print("\nAttempting import: from shared.data_utils import load_flux_from_fits, normalize_flux")
        try:
            from shared.data_utils import load_flux_from_fits, normalize_flux
            print("✓ Import successful!")
        except ImportError as e:
            print(f"✗ Import error: {e}")
            print("\nDetailed traceback:")
            traceback.print_exc()
            
            # Try to get more specific information
            print("\nTrying to diagnose the specific issue:")
            try:
                import shared
                print("  ✓ 'import shared' works")
                try:
                    import shared.data_utils
                    print("  ✓ 'import shared.data_utils' works")
                    print(f"  Available in shared.data_utils: {dir(shared.data_utils)}")
                    
                    # Check if the specific functions exist
                    if hasattr(shared.data_utils, 'load_flux_from_fits'):
                        print("  ✓ 'load_flux_from_fits' exists in shared.data_utils")
                    else:
                        print("  ✗ 'load_flux_from_fits' does NOT exist in shared.data_utils")
                        
                    if hasattr(shared.data_utils, 'normalize_flux'):
                        print("  ✓ 'normalize_flux' exists in shared.data_utils")
                    else:
                        print("  ✗ 'normalize_flux' does NOT exist in shared.data_utils")
                except ImportError as e2:
                    print(f"  ✗ 'import shared.data_utils' fails: {e2}")
            except ImportError as e1:
                print(f"  ✗ 'import shared' fails: {e1}")
    finally:
        # Restore original sys.path
        sys.path = original_path

# Run the debug function
debug_import_with_instrumentation()

## Testing Direct Function Import

Let's try importing the functions directly from their source files to see if that works.

In [ ]:
import sys
import os
import importlib.util

def direct_import_test():
    """Try importing the functions directly from their source files"""
    project_root = os.path.abspath(".")
    fits_loader_path = os.path.join(project_root, "shared", "data_utils", "fits_loader.py")
    
    print(f"Source file exists: {os.path.exists(fits_loader_path)}")
    
    try:
        # Load the module directly from the file
        print(f"Loading module from: {fits_loader_path}")
        spec = importlib.util.spec_from_file_location("fits_loader", fits_loader_path)
        fits_loader = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(fits_loader)
        
        # Check if the functions exist
        print("\nChecking if functions exist in the module:")
        print(f"'load_flux_from_fits' exists: {'load_flux_from_fits' in dir(fits_loader)}")
        print(f"'normalize_flux' exists: {'normalize_flux' in dir(fits_loader)}")
        
        # Try to use the functions
        if 'load_flux_from_fits' in dir(fits_loader) and 'normalize_flux' in dir(fits_loader):
            print("\nSuccessfully imported functions directly from source!")
            return fits_loader.load_flux_from_fits, fits_loader.normalize_flux
        else:
            print("\nFunctions not found in module.")
            return None, None
    except Exception as e:
        print(f"Error importing directly from source: {e}")
        return None, None

# Run the test
load_flux_from_fits_direct, normalize_flux_direct = direct_import_test()

## Creating a Fix for the Import Issue

Based on our investigation, let's create a solution to fix the import issues in the project.

In [ ]:
def create_import_helper():
    """
    Create a helper module that can be used in notebooks to ensure correct imports.
    This will be saved as setup_imports.py in the project root.
    """
    import os
    
    project_root = os.path.abspath(".")
    helper_path = os.path.join(project_root, "setup_imports.py")
    
    helper_code = """
'''
Helper module to ensure correct imports across the project.
Import this at the top of your notebooks to properly set up the Python path.
'''

import os
import sys
import inspect

def setup_project_imports():
    '''
    Add the project root to Python path to enable imports from shared modules.
    This ensures imports work correctly regardless of where the notebook is located.
    '''
    # Get the directory of the calling file (notebook)
    caller_file = inspect.stack()[1].filename
    caller_dir = os.path.dirname(os.path.abspath(caller_file))
    
    # Find the project root by looking for the 'shared' directory
    current_dir = caller_dir
    max_levels = 10  # Prevent infinite loops
    
    for _ in range(max_levels):
        # Check if we've found the project root (contains 'shared' directory)
        if os.path.exists(os.path.join(current_dir, 'shared')):
            project_root = current_dir
            break
        
        # Move up one directory
        parent_dir = os.path.dirname(current_dir)
        if parent_dir == current_dir:  # Reached filesystem root
            break
        current_dir = parent_dir
    else:
        # If we exit the loop normally, we didn't find the project root
        raise ImportError("Could not find project root with 'shared' directory")
    
    # Add project root to path if not already there
    if project_root not in sys.path:
        sys.path.insert(0, project_root)
        print(f"Added {project_root} to Python path")
    
    return project_root

# Run setup automatically when module is imported
project_root = setup_project_imports()
"""
    
    # Write the helper to a file
    with open(helper_path, 'w') as f:
        f.write(helper_code)
    
    print(f"Created import helper at: {helper_path}")
    print("\nTo use this helper in notebooks, add the following code at the top:")
    print("```python")
    print("import sys")
    print("import os")
    print("")
    print("# Add the project root to sys.path")
    print("notebook_dir = os.path.dirname(os.path.abspath('__file__'))")
    print("project_root = os.path.abspath(os.path.join(notebook_dir, '..', '..'))")
    print("if project_root not in sys.path:")
    print("    sys.path.insert(0, project_root)")
    print("")
    print("# Import helper to set up project imports properly")
    print("import setup_imports")
    print("```")
    
    return helper_path

# Create the helper
helper_path = create_import_helper()

## Testing the Import Helper

Let's test our import helper to make sure it solves the problem.

In [ ]:
def test_import_helper():
    """Test that our import helper works correctly"""
    import sys
    import os
    import importlib
    
    # Clear any existing modules
    for module in list(sys.modules.keys()):
        if module in ['shared', 'shared.data_utils', 'setup_imports']:
            del sys.modules[module]
    
    # Store original sys.path
    original_path = sys.path.copy()
    
    try:
        # Import our helper
        print("Importing setup_imports module...")
        import setup_imports
        
        # Now try to import from shared.data_utils
        print("\nTrying import with helper:")
        try:
            from shared.data_utils import load_flux_from_fits, normalize_flux
            print("✓ Import successful!")
            print(f"load_flux_from_fits is callable: {callable(load_flux_from_fits)}")
            print(f"normalize_flux is callable: {callable(normalize_flux)}")
            return True
        except ImportError as e:
            print(f"✗ Import failed: {e}")
            return False
    finally:
        # Restore original sys.path
        sys.path = original_path

# Test the helper
test_successful = test_import_helper()

if test_successful:
    print("\n✅ The import helper successfully fixes the import issue!")
else:
    print("\n❌ The import helper did not fix the issue. More debugging needed.")

## Simulating Fix in Notebook Environment

Now let's simulate applying our fix to the actual notebook environment to verify it works there too.

In [ ]:
def simulate_notebook_with_fix():
    """
    Simulate the notebook environment with our fix applied.
    This changes to the notebook directory and attempts the import with our helper.
    """
    import os
    import sys
    import traceback
    
    # Location of the actual notebook
    project_root = os.path.abspath(".")
    notebook_path = os.path.join(project_root, "projects", "exoplanet-transit-detection", "notebooks", "explore_tess.ipynb")
    notebook_dir = os.path.dirname(notebook_path)
    
    print(f"Notebook directory: {notebook_dir}")
    
    # Store original working directory and sys.path
    original_cwd = os.getcwd()
    original_path = sys.path.copy()
    
    try:
        # Change to the notebook directory
        os.chdir(notebook_dir)
        print(f"Changed working directory to: {os.getcwd()}")
        
        # Clean any existing imports
        for module in list(sys.modules.keys()):
            if module in ['shared', 'shared.data_utils', 'setup_imports']:
                del sys.modules[module]
                
        # Simulate using our helper in the notebook
        print("\nSimulating import code with our helper:")
        print("import sys")
        print("import os")
        print("")
        print("# Add the project root to sys.path")
        print("notebook_dir = os.path.dirname(os.path.abspath('__file__'))")
        print("project_root = os.path.abspath(os.path.join(notebook_dir, '..', '..'))")
        print("if project_root not in sys.path:")
        print("    sys.path.insert(0, project_root)")
        print("")
        print("# Import helper to set up project imports properly")
        print("import setup_imports")
        print("")
        print("# Now import the modules we need")
        print("from shared.data_utils import load_flux_from_fits, normalize_flux")
        
        # Execute the simulated code
        notebook_dir_actual = os.path.dirname(os.path.abspath(notebook_path))
        project_root_actual = os.path.abspath(os.path.join(notebook_dir_actual, '..', '..'))
        if project_root_actual not in sys.path:
            sys.path.insert(0, project_root_actual)
            
        print(f"\nAdded to sys.path: {project_root_actual}")
        
        try:
            import setup_imports
            print("✓ Imported setup_imports")
            
            try:
                from shared.data_utils import load_flux_from_fits, normalize_flux
                print("✓ Successfully imported shared.data_utils functions!")
                return True
            except ImportError as e:
                print(f"✗ Failed to import shared.data_utils functions: {e}")
                traceback.print_exc()
                return False
        except ImportError as e:
            print(f"✗ Failed to import setup_imports: {e}")
            traceback.print_exc()
            return False
    finally:
        # Restore original state
        os.chdir(original_cwd)
        sys.path = original_path
        print(f"\nRestored working directory to: {os.getcwd()}")

# Run the simulation
fix_works_in_notebook = simulate_notebook_with_fix()

if fix_works_in_notebook:
    print("\n✅ The fix works correctly in the notebook environment!")
else:
    print("\n❌ The fix does not work in the notebook environment. Additional debugging needed.")

## Summary and Solution

Based on our extensive debugging, we can provide a solution to fix the import issues in the project.

In [ ]:
def create_summary():
    """
    Provide a summary of our findings and solution
    """
    import os
    
    project_root = os.path.abspath(".")
    
    print("# Import Error Diagnosis and Solution")
    print("\n## Problem Summary")
    print("The import errors in the project were caused by inconsistent Python path configuration.")
    print("When notebooks try to import from shared modules using `from shared.data_utils import ...`,")
    print("the Python interpreter can't find these modules if the project root isn't in sys.path.")
    
    print("\n## Solution Steps")
    print("1. We've created a helper module `setup_imports.py` in the project root that:")
    print("   - Automatically finds the project root directory")
    print("   - Adds it to sys.path if not already present")
    print("   - Can be imported from any notebook or script in the project")
    
    print("\n## How to Use the Solution")
    print("Replace the current import setup in your notebooks with:")
    print("```python")
    print("import sys")
    print("import os")
    print("")
    print("# Add the project root to sys.path (needed to import setup_imports)")
    print("notebook_dir = os.path.dirname(os.path.abspath('__file__'))")
    print("project_root = os.path.abspath(os.path.join(notebook_dir, '..', '..'))")
    print("if project_root not in sys.path:")
    print("    sys.path.insert(0, project_root)")
    print("")
    print("# Import helper to set up project imports properly")
    print("import setup_imports")
    print("")
    print("# Now you can import shared modules without errors")
    print("from shared.data_utils import load_flux_from_fits, normalize_flux")
    print("```")
    
    print("\n## Helper Module Location")
    print(f"The helper module has been created at: {os.path.join(project_root, 'setup_imports.py')}")
    
    print("\n## Additional Recommendations")
    print("1. Consider creating an installable package for shared code:")
    print("   - Update setup.py to include all shared modules")
    print("   - Install in development mode: `pip install -e .`")
    print("   - This eliminates the need for sys.path manipulation")
    print("2. Use absolute imports consistently throughout the project")
    print("3. Create a README.md with instructions for setting up the development environment")

# Generate the summary
create_summary()